In [ ]:
pip install -U nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.6.2)


In [ ]:
!pip install underthesea nltk

In [ ]:
import pickle
import string
import re
from underthesea import sent_tokenize
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, Laplace, KneserNeyInterpolated, WittenBellInterpolated
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/NLP Training/'
# path = ''

In [ ]:
with open(path + 'datasets/real_estates_preprocesed.txt', 'r', encoding='utf-8') as f:
    real_estates_corpus = f.read()
big_corpus = open(path + 'datasets/big_corpus.txt', 'r', encoding='utf-8').read()
big_corpus = big_corpus[:len(big_corpus) // 8]
corpus = real_estates_corpus + big_corpus
len(corpus)

12912152

In [ ]:
sent_corpus = sent_tokenize(corpus) # tách câu
len(sent_corpus)

91007

### Một vấn đề trong xử lý dữ liệu đối với bài toán xử lý ngôn ngữ tự nhiên là phải tokenize. Hiểu nôm na là tách từ, tách câu, nên hàm dưới đây dùng để tách ra

In [ ]:
def tokenize(sent):
    tokens = word_tokenize(sent)
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]
    tokens = [word for word in tokens if word] # remove empty
    return tokens

In [ ]:
word_corpus = [tokenize(sent) for sent in sent_corpus] # tách từng từ của từng câu, tạo thành mảng 2 chiều

In [ ]:
n = 6 # n-gram

In [ ]:
# tách theo n gram
train_data, padded_sents = padded_everygram_pipeline(n, word_corpus)

In [ ]:
vi_model = MLE(n) # tạo mô hình n-gram interpolation
vi_model.fit(train_data, padded_sents)

In [ ]:
with open(path + 'models/n_gram_model.pkl', 'wb') as f:
    pickle.dump(vi_model, f)

In [ ]:
def generate_sent(model, max_num_words, pre_words=[]):
    """
    :param model: An ngram languauge model from `nltk.lm.model`.
    :param max_num_words: Max no. of words to generate.
    :param pre_words: Seed value for random
    """
    content = pre_words
    for i in range(max_num_words):
        token = model.generate(1, text_seed=content[-(n - 1):])
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return content

In [ ]:
detokenize = TreebankWordDetokenizer().detokenize

In [ ]:
def recommend_real_estates_search(txt_input):
#     txt_input = pipeline_preprocessing_corpus(txt_input)
    tokens = word_tokenize(txt_input)
    output_recommend = []
    for i in range(1, 20):
        output = generate_sent(vi_model, i, tokens.copy())
        output_recommend.append(detokenize(output))
    output_recommend = list(set(output_recommend))
    return '\n'.join(output_recommend)

In [ ]:
txt_input = open(path + 'datasets/input.txt', 'r').read() # nếu không muốn nhập đoạn text dưới đây thì nhập vào file input.txt
print(recommend_real_estates_search(txt_input))

bán đất thiên phú tâm huyết công phu khác nhau nhưng điểm
bán đất thiên phú t hà nh phố vũng
bán đất thiên phú thêm
bán đất thiên phú
bán đất thiên phú mỹ biển hồ tràm các quốc lộ 51 56…trong bán kính 2035km hạ tầng giá o thông
bán đất thiên phú rất đáng ngạc nhiên của kho tàng truyền thuyết chuyện kể dân
bán đất thiên phú thịnh phường lô ng bình tân 5x24 full thổ
bán đất thiên phú mỹ biển hồ tràm các quốc lộ 51 56…trong bán kính 2035km hạ tầng giá o thông của
bán đất thiên phú đồng naing ăn g 42m mt bêtôngmt dg đất 48m
bán đất thiên phú và thú vị hơn
bán đất thiên phú chi phí trung
bán đất thiên phú mỹ
bán đất thiên phú linh hoạt
bán đất thiên phú mỹ biển hồ tràm các quốc lộ 51 các cao tốc…trong bán kính 2035km hạ tầng giá o thông
bán đất thiên phú t hà nh phố vũng tàu
